Rename a bunch of files using Git and renamer.

In [16]:
var execSync = require('child_process').execSync;
try {
    require.resolve('renamer');
} catch (e) {
    execSync('npm install renamer');
}

var renamer = require('renamer');
var files = renamer.expand('C:\\Users\\brian.cullinan\\Documents\\portal\\src\\**\\*.scss');
var results = renamer.replace({files: files.filesAndDirs, find: '.scss', replace: '.less'});
renamer.dryRun(results).list.forEach(r => {
    console.log(r.before);
    // rename with git instead
    execSync('git --work-tree="C:\\Users\\brian.cullinan\\Documents\\portal" --git-dir="C:\\Users\\brian.cullinan\\Documents\\portal\\.git" mv ' + JSON.stringify(r.before) + ' ' + JSON.stringify(r.after));
});

Generate a bunch of spec files.

In [33]:
var execSync = require('child_process').execSync;
try {
    require.resolve('glob');
} catch (e) {
    execSync('npm install glob');
}
var fs = require('fs');
var glob = require('glob');
var path = require('path');

var cwd = 'C:\\Users\\brian.cullinan\\Documents\\portal\\src\\app\\subscriptions';

var spec = `
import {ComponentFixture} from '@angular/core/testing';
const assert = require('assert');
import {} from 'jasmine';
import {MockBackend} from '@angular/http/testing';
import {Router} from '@angular/router';
import {Location} from '@angular/common';
import {By} from '@angular/platform-browser';
import {Core} from '../../../testing/core';
{import}

describe('{title}', function () {
    let fixture: ComponentFixture<{component}>;
    let router: Router;
    let location: Location;
    let xhr: MockBackend;
    let comp: {component};

    beforeEach(() => {
        fixture = Core.createAppWithShared([], [], [{component}]);
        comp = fixture.componentInstance;
        router = fixture.debugElement.injector.get(Router);
        xhr = fixture.debugElement.injector.get(MockBackend);
        location = fixture.debugElement.injector.get(Location);
    });

    it('should load {component} component', (done) => {
        fixture.whenStable().then(() => {
            expect(comp).not.toBeNull();
            done();
        });
    });

});
`;

var files = glob.sync('**/*component.ts', {
    ignore: '**/*.spec.ts',
    cwd: cwd
});

var titles = files.map(f => {
    var name = f.split('.')[0].split('/').pop().replace();
    var ucFirst = name.replace(/(^|\s|-)([a-z])/g , (m,p1,p2) => p1+p2.toUpperCase());
    return ucFirst.replace(/-/ig, '');
})

titles.forEach((t, i) => {
    try {
        var specPath = path.join(cwd, files[i].replace('.ts', '.spec.ts'));
        var fileStr = files[i].split('/').pop().replace('.ts', '');
        var importStr = 'import {' + t + 'Component} from \'./' + fileStr + '\';';
        fs.writeFileSync(specPath, spec
                         .replace(/\{component\}/ig, t + 'Component')
                         .replace(/\{title\}/ig, t)
                         .replace(/\{import\}/ig, importStr));
    } catch (e) {}
});


How to use memory-fs and rewire?